#  <span style="text-align=center; color:DarkMagenta; font-family:Georgia; font-size:1.2em;"> **_M2 - TP EIAH - MAY_** </span>

Eliot Lepoittevin & Benjamin Riviere

# <span style="color:brown; font-family:times; font-size:1.3em;">Code</span>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#load de data from the csv files
df_tr = pd.read_csv ('csv_dump/transition.csv')


def LoadPerUserData(dataframe,userNamePos):
    userData = {}
    data = dataframe.values
    
    for i in range(len(data)):
        username = data[i][userNamePos]
        
        if(userData.get(username) == None):
            userData[username] = []
        userData[username].append(data[i])

    return userData

#group the data per user
userData_tr = LoadPerUserData(df_tr,1)
# print(str(userData_tr["mmay"]))

In [2]:
def GetReactivityIndicator(userData_tr):
    indicator = {}
    
    totalResponseTime = 0
    totalMessage = 0
    for d in userData_tr:
        totalResponseTime+=1
        totalMessage+=1
         
    indicator['averageResponseTime']=totalResponseTime/totalMessage
    indicator['test']=3
    
    return indicator


def GetMotivationIndicator(userData_tr):
    indicator = {}
    
    totalResponseTime = 0
    totalMessage = 0
    for d in userData_tr:
        totalResponseTime+=1
        totalMessage+=1
         
    indicator['averageResponseTime']=totalResponseTime/totalMessage
    indicator['test']=3
    
    return indicator


def GenerateUsersIndicators():
    usersIndicators = {}
    
    for userData in userData_tr:
        
        indicators = {}
        indicators['Reactivity']=GetReactivityIndicator(userData)
        indicators['Motivation']=GetMotivationIndicator(userData)
        
        usersIndicators[userData[0]]=indicators
    
    return usersIndicators

In [3]:
usersIndicators = GenerateUsersIndicators()
#print(usersIndicators)

In [4]:
#display specified indicator for specified user
def displayIndicator(indicator,indicatorName):
    #title
    plt.title(indicatorName)
    #labels
    activities = indicator.keys()
    #values
    slices = indicator.values()
    
    def make_autopct(values):
        def my_autopct(pct):
            total = sum(values)
            val = int(round(pct*total/100.0))
            return '{p:.2f}%  ({v:d})'.format(p=pct,v=val)
        return my_autopct
    
    # plotting the pie chart
    plt.pie(slices, labels = activities,
            startangle=90, shadow = True,
            radius = 1.2, autopct=make_autopct(slices))
    # plotting legend
    plt.legend()
    # showing the plot
    plt.show()
    
     
        
displayIndicator(usersIndicators['tdelille']['Reactivity'],'Reactivity')
displayIndicator(usersIndicators['tdelille']['Motivation'],'Motivation')

KeyError: 'tdelille'

# <span style="color:brown; font-family:times; font-size:1.3em;">Résultats</span>

### <span style="color:green; font-family:Georgia; font-size:1.5em;">END</span>

In [ ]:
#Eliot Lepoittevin
#Benjamin Riviere